<a href="https://colab.research.google.com/github/institutohumai/cursos-python/blob/master/DeepLearning/5_Evaluacion_Modelos/ejercicios/ejercicios_solucion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

# Ejercicios de la clase 5

Para llevar adelante los ejercicios de este notebook vamos a recuperar los modelos que construimos en la clase 3 para clasificación sobre FashionMNIST.

In [1]:
import torch
from torch import nn

INPUT = 28 * 28 
OUTPUT = 10 

net1 = nn.Sequential(nn.Flatten(),
                    nn.Linear(INPUT, 512),
                    nn.ReLU(),
                    nn.Linear(512, 128),
                    nn.ReLU(),
                    nn.Linear(128, OUTPUT))

net2 = nn.Sequential(nn.Flatten(),
                    nn.Linear(INPUT, 512),
                    nn.Sigmoid(),
                    nn.Linear(512, 128),
                    nn.Sigmoid(),
                    nn.Linear(128, OUTPUT))

net3 = nn.Sequential(nn.Flatten(),
                    nn.Linear(INPUT, 1024),
                    nn.ReLU(),
                    nn.Linear(1024, 1024),
                    nn.ReLU(),
                    nn.Linear(1024, OUTPUT))

Ahora repetiremos la evaluación que hicimos en esa clase, pero llevándola adelante de manera más exhaustiva con K-fold Cross Validation. Para eso, cargaremos solo los datos de testeo de FasionMNIST y fingiremos que esos 10000 ejemplos son todos los que tenemos.

In [2]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

data_iter = datasets.FashionMNIST(
        root="../data", train=False, transform=transforms.ToTensor(), download=True)

## Ejercicio 
Verifique cuál de los modelos anteriores es el mejor llevando adelante proceso de cross validation con 3 folds y entrenando por 20 epochs. Reutilice todas las funciones que necesite de los notebooks de teoría de la clase 5 y de todos los ejercicios anteriores.

Nota: le recomendamos que guarde las accuracy tanto de testeo como de entrenamiento porque le servirán para más adelante.

In [3]:
#### Funciones necesarias de otros notebooks

def reset_weights(m):
  if type(m) == nn.Linear:
      nn.init.normal_(m.weight, std=0.01)

def accuracy(y_hat, y):
    """Compute the number of correct predictions."""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())

    
def test_accuracy(fold,model, loss, device, test_loader):
  # inserte su código aquí
  TestAcc = 0.0
  N = 0
  for X, y in test_loader:
      X, y = X.to(device), y.to(device)
      N += y.numel()
      TestAcc += accuracy(model(X), y)
  print('\nTest set for fold {}:  Accuracy: {}/{} ({:.0f}%)'.format(
        fold, TestAcc, N,
        (100. * TestAcc) / N))
  return TestAcc / N

def train_accuracy(fold,model, loss, device, test_loader):
  # inserte su código aquí
  TestAcc = 0.0
  N = 0
  for X, y in test_loader:
      X, y = X.to(device), y.to(device)
      N += y.numel()
      TestAcc += accuracy(model(X), y)
  print('\nTrain set for fold {}:  Accuracy: {}/{} ({:.0f}%)'.format(
        fold, TestAcc, N,
        (100. * TestAcc) / N))
  return TestAcc / N

def train(fold, model, device, loss, train_loader, optimizer, epoch):

  for batch_idx, (data, target) in enumerate(train_loader):
      data, target = data.to(device), target.to(device)
      optimizer.zero_grad()
      l = loss(model(data), target).mean()
      l.backward()
      optimizer.step()

In [4]:
from sklearn.model_selection import KFold

### Función que lleva adelante el proceso de kfold cross validation
def train_kfold(model, dataset, n_fold, epochs):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  loss = torch.nn.CrossEntropyLoss(reduction='none')
  optimizer = torch.optim.Adam(model.parameters())
  batch_size=32
  folds=n_fold
  train_acc = []
  acc = []
  kfold=KFold(n_splits=n_fold,shuffle=True)
  for fold,(train_idx,test_idx) in enumerate(kfold.split(dataset)):
    print('------------fold no---------{}----------------------'.format(fold))
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_idx)

    trainloader = torch.utils.data.DataLoader(
                        dataset, 
                        batch_size=batch_size, sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                        dataset,
                        batch_size=batch_size, sampler=test_subsampler)

    model.apply(reset_weights)

    fold_acc = 0
    for epoch in range(1, epochs + 1):
      train(fold, model, device, loss, trainloader, optimizer, epoch)
      fold_train_acc = train_accuracy(fold,model, loss, device, trainloader)
      fold_acc = test_accuracy(fold,model, loss, device,  testloader)
    train_acc.append(fold_train_acc)
    acc.append(fold_acc)
  return train_acc, acc

In [ ]:
##### Lleva adelante los entrenamientos
train_acc1, acc_1 = train_kfold(net1, data_iter, 3, 20)
train_acc2, acc_2 = train_kfold(net2, data_iter, 3, 20)
train_acc3, acc_3 = train_kfold(net3, data_iter, 3, 20)


In [7]:
import numpy as np
print("Modelo 1:  entrenamiento ", np.array(train_acc1).mean(), ", validación: ", np.array(acc_1).mean() )
print("Modelo 2:  entrenamiento ", np.array(train_acc2).mean(), ", validación: ", np.array(acc_2).mean() )
print("Modelo 3:  entrenamiento ", np.array(train_acc3).mean(), ", validación: ", np.array(acc_3).mean() )


Modelo 1:  entrenamiento  0.9299502546624844 , validación:  0.8491994370402848
Modelo 2:  entrenamiento  0.91875006459698 , validación:  0.8403005267533352
Modelo 3:  entrenamiento  0.9470998847297359 , validación:  0.8509007169463197


In [9]:
print("Modelo 3:  entrenamiento ", np.array(train_acc3).mean(), ", validación: ", np.array(acc_3).mean() )

Modelo 3:  entrenamiento  0.8948503519918513 , validación:  0.8312986067652957
